In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.1 MB/s eta 0:00:0000:0100:08
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 1.1 MB/s eta 0:00:00a 0:00:01m
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=81192f99977b0368e8e702719f0b17c09806d8a7e83875b2d6247541f60f4616
  Stored in directory: /Users/jabama/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [4]:
!sudo add-apt-repository ppa:webupd8team/java


Password:
sudo: a password is required


In [3]:
spark = SparkSession.builder.appName("churn_pred").getOrCreate()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/opt/anaconda3/lib/python3.11/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [4]:
df = spark.read.csv("../input/generalassemblywelcome5k/customer_churn.csv",header=True , inferSchema=True)

In [5]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [6]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                null|                null|0.16666666666666666|
| stddev|         null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [7]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
df.dtypes

[('Names', 'string'),
 ('Age', 'double'),
 ('Total_Purchase', 'double'),
 ('Account_Manager', 'int'),
 ('Years', 'double'),
 ('Num_Sites', 'double'),
 ('Onboard_date', 'timestamp'),
 ('Location', 'string'),
 ('Company', 'string'),
 ('Churn', 'int')]

In [9]:
from pyspark.ml.feature import VectorAssembler

In [12]:
assembler = VectorAssembler(inputCols=[
#  'Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
#  'Onboard_date',
#  'Location',
#  'Company',
#  'Churn'
 ],
 outputCol="features")

In [13]:
output = assembler.transform(df)

In [14]:
df_final = output.select("features","churn")

In [15]:
df_final.show()

+--------------------+-----+
|            features|churn|
+--------------------+-----+
|[42.0,11066.8,0.0...|    1|
|[41.0,11916.22,0....|    1|
|[38.0,12884.75,0....|    1|
|[42.0,8010.76,0.0...|    1|
|[37.0,9191.58,0.0...|    1|
|[48.0,10356.02,0....|    1|
|[44.0,11331.58,1....|    1|
|[32.0,9885.12,1.0...|    1|
|[43.0,14062.6,1.0...|    1|
|[40.0,8066.94,1.0...|    1|
|[30.0,11575.37,1....|    1|
|[45.0,8771.02,1.0...|    1|
|[45.0,8988.67,1.0...|    1|
|[40.0,8283.32,1.0...|    1|
|[41.0,6569.87,1.0...|    1|
|[38.0,10494.82,1....|    1|
|[45.0,8213.41,1.0...|    1|
|[43.0,11226.88,0....|    1|
|[53.0,5515.09,0.0...|    1|
|[46.0,8046.4,1.0,...|    1|
+--------------------+-----+
only showing top 20 rows



In [16]:
train , test = df_final.randomSplit([0.8,0.2])

In [17]:
from pyspark.ml.classification import LogisticRegression

In [18]:
lr = LogisticRegression(labelCol="churn")

In [19]:
lrm = lr.fit(train)

In [20]:
lrm.summary

In [21]:
lrm.summary.accuracy

0.8908839779005525

In [22]:
lrm.summary.areaUnderROC

0.8985346204597507

In [23]:
lrm.summary.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|  0.0|[4.52257618166588...|[0.98925568640428...|       0.0|
|[25.0,9672.03,0.0...|  0.0|[4.66657435493585...|[0.99068318865547...|       0.0|
|[26.0,8787.39,1.0...|  1.0|[0.68234512062331...|[0.66426190288573...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.28075600541315...|[0.99813151295094...|       0.0|
|[27.0,8628.8,1.0,...|  0.0|[5.29578332832912...|[0.99501231543981...|       0.0|
|[28.0,11128.95,1....|  0.0|[4.06607205818483...|[0.98314438338493...|       0.0|
|[28.0,11204.23,0....|  0.0|[1.90124029245322...|[0.87003183786887...|       0.0|
|[28.0,11245.38,0....|  0.0|[3.76975108403600...|[0.97746187753783...|       0.0|
|[29.0,5900.78,1.0...|  0.0|[4.00073196490847...|[0.98202671395868...|       0.0|
|[29.0,8688.17,1

In [24]:
lrm.summary.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               724|                724|
|   mean|0.1643646408839779|0.12154696132596685|
| stddev|0.3708623417590445| 0.3269877343690762|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [25]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [26]:
pred_labels = lrm.evaluate(test)

In [27]:
pred_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[28.0,8670.98,0.0...|    0|[7.60382163750703...|[0.99950170588146...|       0.0|
|[28.0,9090.43,1.0...|    0|[1.52847413391011...|[0.82178295209264...|       0.0|
|[29.0,9617.59,0.0...|    0|[4.41641307056050...|[0.98806664902087...|       0.0|
|[30.0,7960.64,1.0...|    1|[3.02076652378059...|[0.95350352079305...|       0.0|
|[30.0,10183.98,1....|    0|[2.81851952338095...|[0.94366841842908...|       0.0|
|[30.0,12788.37,0....|    0|[2.52555788533945...|[0.92591421321065...|       0.0|
|[31.0,5304.6,0.0,...|    0|[3.42909968551911...|[0.96860169853297...|       0.0|
|[31.0,7073.61,0.0...|    0|[3.12378420423470...|[0.95786322833749...|       0.0|
|[31.0,9574.89,0.0...|    0|[3.33048590542651...|[0.96545997687205...|       0.0|
|[31.0,10058.87,

In [28]:
eval = BinaryClassificationEvaluator(rawPredictionCol="prediction" , labelCol='churn')

In [29]:
auc = eval.evaluate(pred_labels.predictions)

In [30]:
auc

0.7926585094549498